# Latent Space Mapping: Brain Features → Music

This notebook demonstrates strategies for mapping neural features to music model latent spaces.

## Approaches
1. **Linear Mapping** - Simple projections from brain to latent space
2. **Neural Network Mapping** - Learn nonlinear transformations
3. **Semantic Conditioning** - Use text/CLAP embeddings as intermediary
4. **Direct Parameter Control** - Map to interpretable synthesis parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

print("Libraries loaded successfully")

## 1. Simulate Data

Create synthetic brain features and music latent vectors for demonstration.

In [ ]:
# Simulate brain features (like EEG or fMRI features)
n_samples = 200
n_brain_features = 10  # e.g., band powers, connectivity, etc.
n_latent_dims = 8  # Target music latent space dimensions

# Generate synthetic brain features
brain_features = np.random.randn(n_samples, n_brain_features)

# Generate synthetic music latent vectors
# In reality, these would come from a pretrained music model (MusicVAE, etc.)
music_latents = np.random.randn(n_samples, n_latent_dims)

# Create a synthetic relationship (for demonstration)
# Real data would come from paired brain-music recordings
true_mapping = np.random.randn(n_brain_features, n_latent_dims) * 0.5
music_latents = brain_features @ true_mapping + 0.3 * np.random.randn(n_samples, n_latent_dims)

print(f"Brain features shape: {brain_features.shape}")
print(f"Music latents shape: {music_latents.shape}")

# Split into train/test
split_idx = int(0.8 * n_samples)
X_train, X_test = brain_features[:split_idx], brain_features[split_idx:]
y_train, y_test = music_latents[:split_idx], music_latents[split_idx:]

## 2. Linear Mapping

Simple ridge regression to map brain features to music latent space.

In [ ]:
# Standardize features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Train ridge regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train_scaled)

# Predict
y_pred_scaled = ridge_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Evaluate
mse = np.mean((y_test - y_pred) ** 2)
r2 = ridge_model.score(X_test_scaled, y_test_scaled)

print(f"Linear Mapping Performance:")
print(f"  MSE: {mse:.4f}")
print(f"  R²: {r2:.4f}")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 4, figsize=(16, 6))
axes = axes.flatten()

for dim in range(n_latent_dims):
    axes[dim].scatter(y_test[:, dim], y_pred[:, dim], alpha=0.6)
    axes[dim].plot([y_test[:, dim].min(), y_test[:, dim].max()],
                   [y_test[:, dim].min(), y_test[:, dim].max()],
                   'r--', linewidth=2)
    axes[dim].set_xlabel('True Latent')
    axes[dim].set_ylabel('Predicted Latent')
    axes[dim].set_title(f'Dimension {dim+1}')
    axes[dim].grid(alpha=0.3)

plt.tight_layout()
plt.suptitle('Linear Mapping: True vs Predicted Latent Dimensions', y=1.02, fontsize=14)
plt.show()

## 3. Neural Network Mapping

Learn a nonlinear mapping using a small neural network.

In [ ]:
class BrainToMusicMapper(nn.Module):
    """Neural network for brain → music latent mapping"""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# Create model
model = BrainToMusicMapper(
    input_dim=n_brain_features,
    hidden_dim=32,
    output_dim=n_latent_dims
)

print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# Convert to PyTorch tensors
X_train_t = torch.FloatTensor(X_train_scaled)
y_train_t = torch.FloatTensor(y_train_scaled)
X_test_t = torch.FloatTensor(X_test_scaled)
y_test_t = torch.FloatTensor(y_test_scaled)

# Training setup
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
n_epochs = 200

# Training loop
losses = []
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    y_pred_t = model(X_train_t)
    loss = criterion(y_pred_t, y_train_t)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

print("\n✓ Training complete")

In [ ]:
# Evaluate neural network
model.eval()
with torch.no_grad():
    y_pred_nn_t = model(X_test_t)
    test_loss = criterion(y_pred_nn_t, y_test_t).item()
    
y_pred_nn = scaler_y.inverse_transform(y_pred_nn_t.numpy())

mse_nn = np.mean((y_test - y_pred_nn) ** 2)

print(f"Neural Network Mapping Performance:")
print(f"  MSE: {mse_nn:.4f}")
print(f"  Test Loss: {test_loss:.4f}")

# Plot training curve
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Comparison: Linear vs Neural Network

In [ ]:
# Compare methods
comparison = {
    'Method': ['Linear (Ridge)', 'Neural Network'],
    'MSE': [mse, mse_nn],
    'Complexity': ['Low', 'Medium'],
    'Real-time': ['✓ Fast', '✓ Fast'],
    'Interpretability': ['High', 'Low']
}

print("\nMethod Comparison:")
print("="*60)
for key in comparison.keys():
    print(f"{key:20s} {str(comparison[key][0]):20s} {str(comparison[key][1]):20s}")
print("="*60)

## 5. Semantic Conditioning Strategy

Alternative: Map brain states to semantic descriptions, then use text-to-music models.

In [ ]:
# Define semantic music dimensions
semantic_dims = [
    ('energy', ['calm', 'moderate', 'energetic']),
    ('valence', ['sad', 'neutral', 'happy']),
    ('complexity', ['simple', 'moderate', 'complex']),
    ('tempo', ['slow', 'medium', 'fast'])
]

print("Semantic Conditioning Approach:")
print("\nBrain Features → Semantic Attributes → Text Description → Music")
print("\nExample mappings:")
print("  High engagement + positive asymmetry → 'energetic happy music'")
print("  High theta/alpha + low engagement → 'calm contemplative ambient'")
print("  High beta + negative asymmetry → 'tense focused electronic'")

# Example: Construct text description from features
def brain_to_text_description(brain_features):
    """Convert brain features to music description"""
    # Simplified example
    engagement = brain_features[0]
    valence = brain_features[1]
    complexity = brain_features[2]
    
    energy = 'energetic' if engagement > 0.5 else 'calm'
    mood = 'happy' if valence > 0 else 'melancholic'
    texture = 'complex' if complexity > 0.5 else 'simple'
    
    return f"{energy} {mood} music with {texture} harmonies"

# Example
example_features = np.array([0.8, 0.3, -0.2])
description = brain_to_text_description(example_features)
print(f"\nExample: Features {example_features} → '{description}'")

## 6. Direct Parameter Control (DDSP-style)

Map brain features to interpretable synthesis parameters.

In [ ]:
# Define interpretable audio parameters
audio_params = {
    'fundamental_frequency': (100, 500),  # Hz
    'loudness': (0.1, 1.0),  # Amplitude
    'harmonic_distribution': (0, 1),  # Brightness
    'noise_amount': (0, 0.5),  # Texture
    'reverb': (0, 1),  # Spaciousness
    'attack_time': (0.01, 0.5)  # Seconds
}

print("Direct Parameter Mapping:")
print("\nBrain Feature → Audio Parameter:")
print("  Engagement → Tempo/Energy")
print("  Frontal Asymmetry → Harmonic Mode (major/minor)")
print("  Theta/Alpha Ratio → Harmonic Complexity")
print("  Overall Activation → Loudness")
print("  DMN Connectivity → Reverb/Spaciousness")

def map_brain_to_audio_params(brain_features):
    """Map brain features to audio synthesis parameters"""
    params = {}
    
    # Example mappings (would be learned or designed)
    params['fundamental_frequency'] = 200 + 200 * brain_features[0]  # Engagement → pitch
    params['loudness'] = 0.3 + 0.5 * brain_features[1]  # Activation → volume
    params['harmonic_distribution'] = (brain_features[2] + 1) / 2  # Normalize to [0, 1]
    params['noise_amount'] = max(0, brain_features[3] * 0.3)
    params['reverb'] = (brain_features[4] + 1) / 2
    params['attack_time'] = 0.01 + 0.2 * (1 - brain_features[0])  # Fast attack when engaged
    
    return params

# Example
example_brain = np.random.randn(5)
audio_params_out = map_brain_to_audio_params(example_brain)

print("\nExample audio parameters:")
for param, value in audio_params_out.items():
    print(f"  {param}: {value:.3f}")

## Summary and Recommendations

### Mapping Strategies Compared

| Strategy | Pros | Cons | Best For |
|----------|------|------|----------|
| **Linear Mapping** | Simple, interpretable, fast | Limited expressiveness | Initial prototyping |
| **Neural Network** | Captures nonlinear relationships | Needs training data, less interpretable | Performance optimization |
| **Semantic Conditioning** | Leverages pretrained models, intuitive | Indirect control | Text-to-music models |
| **Direct Parameters** | Highly interpretable, real-time | Requires parameter design | DDSP-like synthesis |

### Recommended Approach

1. **Start with direct parameter mapping**
   - Design interpretable mappings
   - Test with users for intuitive feel
   - Iterate based on feedback

2. **Add semantic layer**
   - Map brain states to high-level descriptors
   - Use for text-to-music conditioning
   - Provides user-understandable explanations

3. **Optimize with learning**
   - Collect paired brain-music preferences
   - Train neural network for personalization
   - Keep interpretable fallback

### Key Considerations

- **Agency**: Users should understand and control the mapping
- **Latency**: Real-time requires <100ms end-to-end
- **Personalization**: Individual differences necessitate calibration
- **Stability**: Mappings should be smooth and predictable
- **Expressiveness**: Balance control and autonomy

### Next Steps

1. Test mappings with real neural data
2. Implement in real-time system (see `toy_interface/`)
3. Conduct user studies for validation
4. Iterate based on user experience